In [40]:
import boto3
import json
s3 = boto3.client('s3')
bucket1 = 'bootcampjanuary2018-1a'
bucket2 = 'bootcampaugust2017-1b'

In [14]:
# delete a file
s3.delete_object(Bucket=bucket1, Key='note.txt')

{'ResponseMetadata': {'HTTPHeaders': {'date': 'Mon, 12 Feb 2018 14:52:28 GMT',
   'server': 'AmazonS3',
   'x-amz-id-2': 'yT96RmNYh5CWU9p3znwFNDegc6VmBxkASYHkLfWervr67gtnYCq5Thu/56accdKY5qGOecrQKrQ=',
   'x-amz-request-id': '9FFE8CA2825FF701'},
  'HTTPStatusCode': 204,
  'HostId': 'yT96RmNYh5CWU9p3znwFNDegc6VmBxkASYHkLfWervr67gtnYCq5Thu/56accdKY5qGOecrQKrQ=',
  'RequestId': '9FFE8CA2825FF701',
  'RetryAttempts': 0}}

In [24]:
# upload 

s3.upload_file('2007_clean.csv', bucket1, '2007_clean.csv')

In [84]:
# download
obj = s3.get_object(Bucket=bucket1, Key='table_data.json')
file_str = json.loads(obj['Body'].read().decode("utf-8"))

In [86]:
file_str

{'2007_Data': 603,
 '2008_Data': 2393,
 '2009_Data': 5281,
 '2010_Data': 12537,
 '2011_Data': 21721,
 '2012_Data': 53367,
 '2013_Data': 134756,
 '2014_Data': 235629,
 'Member_Information': 466287}

In [36]:
def moveToDeprecated(table_name):
    deprecated_path = 'deprecated/{}_{}.csv'.format(table_name, datetime.now().strftime('%Y_%m_%d'))
    current_path = '{}.csv'.format(table_name)
    try:
        s3.copy_object(Bucket=bucket1, CopySource=bucket1+'/'+current_path, Key=deprecated_path)
        s3.delete_object(Bucket=bucket1, Key=current_path)
    except Exception as e:
        print(e)

In [37]:
from datetime import datetime
moveToDeprecated('2007_clean')

An error occurred (NoSuchKey) when calling the CopyObject operation: The specified key does not exist.


In [48]:
from io import BytesIO
import pandas as pd
def s3ToDataFrame(table_name):
    obj = s3.get_object(Bucket=bucket1, Key='data/{}.csv'.format(table_name))
    io = BytesIO(obj['Body'].read())
    return pd.read_csv(io)

In [69]:
df = s3ToDataFrame('2007_Data') ###check row count mis match

In [68]:
s3ToDataFrame('2007_Data')['pub_rec_zero'].unique()

array([1., 0.])

In [70]:
df.head()

,Unnamed: 0,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,total_acc,deliquency,status_binary,region,return,profit
0,5,70694,5000,5000,350,36,11.54,164.98,C,C5,...,4.0,NaN,6.0,0.0,13.0,0,1,SE,0.300238,-3498.81
1,13,76583,5000,5000,1775,36,9.01,159.03,B,B2,...,0.0,5.0,5.0,0.0,7.0,1,0,NE,1.144964,724.82
2,26,84914,5000,5000,2025,36,7.43,155.38,A,A2,...,1.0,NaN,12.0,0.0,21.0,0,0,NE,1.040088,200.44
3,31,86999,7500,7500,800,36,13.75,255.43,E,E2,...,0.0,11.0,7.0,0.0,8.0,1,0,NE,1.225997,1694.98
4,32,88023,4400,4400,1400,36,9.64,141.25,B,B4,...,0.0,0.0,4.0,0.0,5.0,0,0,NE,1.155561,684.47
